In [2]:
import numpy as np
import matplotlib.pyplot as plt
import time
from tqdm import tqdm
import tensorflow as tf
from numba import jit
from keras.datasets import mnist
from sklearn import datasets
from sklearn.model_selection import train_test_split

In [18]:
use_mnist = False

# Gaussian Blob Data

In [19]:
if not use_mnist:
    x_dim = 2 # dimension of observations
    num_clusters = 10
    mus = []

    sig = np.random.randn(x_dim,x_dim)/np.sqrt(x_dim)
    Sigma = sig@sig.T

    for i in range(num_clusters):
        mus.append(10*np.random.randn(x_dim,1))

    per_class_sample = 1000 # number of data points
    samples = per_class_sample * num_clusters

    classes = []
    for i in range(num_clusters):
        classes.append(mus[i] + np.random.randn(x_dim,int(per_class_sample)))

    X = np.concatenate(classes, axis=1)

# MNIST 64x64

In [20]:
full = False
if use_mnist:
    if full:
        (train_X, train_y), (test_X, test_y) = mnist.load_data()
        train_X = train_X.reshape(-1,784).astype(np.float64)
        test_X = test_X.reshape(-1,784).astype(np.float64)
    if not full:
        digits = datasets.load_digits()
        train_X, test_X, train_y, test_y = train_test_split(digits.data, digits.target, test_size=0.4, random_state=4)
        train_X = train_X.reshape(-1,64).astype(np.float64)
        test_X = test_X.reshape(-1,64).astype(np.float64)
    samples = train_X.shape[0]
    x_dim = train_X.shape[1]
    X = train_X.T

# Hard Clustering (Neural + Non-Neural)

In [21]:
if use_mnist:
    num_clusters = 100
else:
    num_clusters = 10

In [22]:
neural = False

In [23]:
print(num_clusters)
print(samples)
print(x_dim)
print(neural)
print(X.shape)

10
10000
2
False
(2, 10000)


In [24]:
W = np.random.randn(num_clusters,x_dim)
theta = np.random.randn(num_clusters,1)
n = np.ones((num_clusters,1))
Y = np.zeros((num_clusters,samples))
eta = .1
rho = 5



#idx = np.random.permutation(samples)

#s = np.empty(idx.size, dtype=np.int32)
#for i in np.arange(idx.size):
#    s[idx[i]] = i
    
    
#X = X[:, idx]
#Z = Z[:, idx]

err = [[] for _ in range(samples)]
    
    
W = 0.5 * np.random.randn(num_clusters,x_dim)
theta = np.random.randn(num_clusters,1)
n = np.ones((num_clusters,1))
Y = np.zeros((num_clusters,samples)) 


for t in tqdm(range(samples)):
    x = X[:,t].reshape((x_dim,1))
    a = -W@x + theta
    y = np.zeros((num_clusters,1))
    z = np.zeros(1)

    if neural: ## Neural Step 3

        er  = 1
        itr = 1

        # Iterate until convergence
        while er > 1e-7:

            z_prev = z
            y_prev = y


            # Update y and z
            y = tf.nn.relu(y + (eta)*( -a - z - rho*(np.sum(y, axis=0) - 1) )).numpy()
            z += eta*( np.sum(y, axis=0) - 1 )


            er = max(np.linalg.norm(z_prev-z)/(np.linalg.norm(z_prev)+1e-4),
                     np.linalg.norm(y_prev-y)/(np.linalg.norm(y_prev)+1e-4))
            err[t].append(er)
            itr += 1


    else: ## Non=Neural Step 3
        c = np.argmin(a)
        y[c] = 1
        z = -a[c]


    #z = z[0]

    #Step 4

    n += y

    for i in range(num_clusters):
        for j in range(x_dim):
            W[i,j] += y[i]*(2*x[j] - W[i,j])/min(n[i], 100)

        theta[i] += y[i]*(z - theta[i])/min(n[i], 100)

    for j in range(num_clusters):
        Y[j,t] = y[j]

#print(f'Cluster {1} mean: {W[0,:]/2}')
#for i in range(num_clusters):
#    print(f'Cluster {i+1} mean: {W[i,:]/2}')
#print()

100%|███████████████████████████████████| 10000/10000 [00:02<00:00, 3898.55it/s]


In [25]:
Y.shape

(10, 10000)

In [26]:
np.argmax(Y, axis=0)

array([3, 3, 3, ..., 0, 0, 0])

In [27]:
np.mean(np.argmax(Y, axis=0))

3.8428